In [153]:
import pandas as pd
import datetime as dt

#grab flight plan
flight_plan = pd.read_csv(r'C:\Users\cicic\OneDrive\Desktop\Barrios_Project\barrios\iss-data\csv\iss_flight_plan_20220101-20251231.csv')

#query to only use dates where the event is dock and convert it to a list
dock_days_list = flight_plan[flight_plan['event'] == 'Dock']['datedim'].to_list()

#convert list of date strings into a list of datetime objects
dates_list = [dt.datetime.strptime(date, '%m/%d/%Y').date() for date in dock_days_list]

#print the days difference between each date
num_days = pd.Series(dates_list).diff().dt.days.iloc[1:].astype(int).tolist()
print(num_days)




[4, 25, 22, 18, 24, 13, 43, 67, 15, 22, 12, 18, 76, 15, 5, 13, 67, 2, 13, 59, 21, 2, 19, 48, 29, 41, 10, 10, 17, 2, 15, 8, 30, 22, 7, 22, 29, 35, 16, 6, 14, 9, 44, 33, 4, 22, 1, 34, 3, 19, 17, 62, 4, 27, 48, 2, 29, 7, 25, 28, 25, 25]


In [154]:
#grab crew members per dock
crewF = pd.read_csv(r'C:\Users\cicic\OneDrive\Desktop\Barrios_Project\barrios\iss-data\csv\iss_flight_plan_crew_20220101-20251321.csv')

crew_per_dock = []

for i, date in enumerate(dock_days_list):
    crewNum = crewF[crewF['datedim'] == date]['crew_count'].tolist()
    Amount = sum(crewNum)
    crew_per_dock.append(Amount)

#number of crew members on board during each dock
print(crew_per_dock)


[7, 7, 10, 11, 11, 7, 7, 7, 10, 11, 7, 7, 7, 7, 7, 11, 7, 11, 11, 7, 7, 7, 11, 10, 7, 7, 11, 11, 7, 7, 11, 7, 10, 9, 7, 11, 7, 7, 11, 7, 7, 10, 10, 11, 7, 7, 7, 7, 7, 11, 7, 10, 11, 11, 7, 7, 11, 7, 10, 7, 7, 11, 7]


In [155]:
#grab rates
rater = pd.read_csv(r'C:\Users\cicic\OneDrive\Desktop\Barrios_Project\barrios\iss-data\csv\rates_definition.csv')


#consumable must match a consumable on the rates csv. case sensitive.
consumable = 'Pretreat Tanks'
consumable = 'Water'


#sum of generated
sum_generated = sum(rater[(rater['affected_consumable'] == consumable) & (rater['type'] == 'generation')]['rate'].to_list())

#grabs the usage rate of consumable and sums it up
sum_usage = sum(rater[(rater['affected_consumable'] == consumable) & (rater['type'] == 'usage')]['rate'].to_list())

#if units contains "/crew" then do something else
crew_check = rater[rater['affected_consumable'] == consumable]['units'].tolist()
combine =  '\t'.join(crew_check)
print(combine)



#prints
print("Consumable used = ", sum_usage)
print("Consumable generated = ", sum_generated)
net_consumable = sum_generated - sum_usage
print("net consumable per day= ", net_consumable)



Liters/Day	Liters/Crew/Day	Liters/Crew/Day	Liters/Day	Liters/Day	Liters/Day	Liters/Day
Consumable used =  16.84
Consumable generated =  23.549999999999997
net consumable per day=  6.709999999999997


In [156]:
import math
rate = -(net_consumable)

raw_to_send = [i * rate for i in num_days]
consumable_to_send = []
current_sum = 0

if(1==1): #if crewcheck != /crew
    #if the rate of depletion is less than 1 unit per day then we send just 1 unit on certain shipments
    if rate < 1 and rate > 0:
        for i, value in enumerate(raw_to_send):
            current_sum+= value
            if current_sum < 1:
                consumable_to_send.append(0)
            else:
                consumable_to_send.append(1)
                current_sum = 0
        print(consumable_to_send)
    else:
        #otherwise we take the raw and round up
        consumable_to_send = [math.ceil(i * rate) for i in num_days]
        print(consumable_to_send)
else: #do some crazy work
    print ("jello")


[-26, -167, -147, -120, -161, -87, -288, -449, -100, -147, -80, -120, -509, -100, -33, -87, -449, -13, -87, -395, -140, -13, -127, -322, -194, -275, -67, -67, -114, -13, -100, -53, -201, -147, -46, -147, -194, -234, -107, -40, -93, -60, -295, -221, -26, -147, -6, -228, -20, -127, -114, -416, -26, -181, -322, -13, -194, -46, -167, -187, -167, -167]


In [157]:
##RUN ALL or this wont work
dock_days_list.pop()

readable = pd.DataFrame(
    {'Docking Days': dock_days_list,
     'Amount of Consumable to Send': consumable_to_send
    })

display(readable)

,Docking Days,Amount of Consumable to Send
0,2/17/2022,-26
1,2/21/2022,-167
2,3/18/2022,-147
3,4/9/2022,-120
4,4/27/2022,-161
...,...,...
57,9/3/2025,-46
58,9/10/2025,-167
59,10/5/2025,-187
60,11/2/2025,-167
